In [60]:
import benepar, spacy
benepar.download('benepar_en3')


[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /Users/kostadindev/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [61]:
#list(sent._.children)[3]
#print(list(sent._.children)[2].label)
# for i in range(len(list(sent._.children))):
#     print(list(sent._.children[i]), list(sent._._labels)[i])
# list(sent._.children)
import pandas as pd
from pprint import pprint as pprint
import numpy as np
from sklearn.cluster import AffinityPropagation, KMeans
import distance

In [62]:
def pd_print_entry(df, index):
    entry = df.iloc[index]
    print("Sentence: ")
    print(entry['sentence'])
    print("Questions: ")
    print(entry['questions'])
    print('Answers')
    #print(entry['answers'])
    for answer in entry['answers']:
        print(answer)
    print('Removed')
    #print(entry['answers'])
    for removed in entry['removed']:
        print(removed)
    print('Phrases: ')
    pprint(entry['phrases'])

In [63]:
def get_phrases(sent, recursive = False):
    children = list(sent._.children)
    #VP, NP, SBAR, ADVP, PP = [], [], [], [], []
    phrases = {}
    for i in range(len(children)):
        if str(children[i]) not in signs:
            try:
                label = children[i]._.labels[0]
            except:
                label= "stop"
            # print(children[i], label)
            # print(children[i], children[i]._.labels)
            # generate_questions(children[i])
            if label != 'stop':
                if label not in phrases.keys():
                    phrases[label] = []
                phrases[label].append(children[i])
                new_phrases= get_phrases(children[i], recursive= recursive)
                for key in new_phrases.keys():
                    if key not in phrases.keys():
                        phrases[key] = []
                    phrases[key].extend(new_phrases[key])
    return phrases

In [64]:
def get_QA(phrases):
    questions, answers = [], []
    if 'SBAR' in phrases.keys():
        for sbar in phrases['SBAR']:
            questions.append(f"What {str(sbar)}?")
            if 'NP' in phrases.keys():
                for np in phrases['NP']:
                    if 'VP' in phrases.keys():
                        for vp in phrases['VP']:
                            answers.append(f'<np>{np}<np> <vp>{vp}<vp>')
    return questions, answers

In [65]:
def edit_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [66]:
# TODO REVISIT USING TREE
def filter_edit_distance(words):
    if len(words) == 0:
        return []
    filtered_words = []
    words = np.asarray(words) #So that indexing with a list will work
    lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])
    try:
        clustering = AffinityPropagation(affinity="precomputed", damping=0.5)
        clustering.fit(lev_similarity)
        for cluster_id in np.unique(clustering.labels_):
            exemplar = words[clustering.cluster_centers_indices_[cluster_id]]
            filtered_words.append(exemplar)
    except:
        clustering = KMeans(n_clusters=max(int(np.sqrt(len(words))),5), random_state=0).fit(lev_similarity)
        for cluster_id in np.unique(clustering.labels_):
            exemplar = np.random.choice(words[clustering.labels_==cluster_id])
            filtered_words.append(exemplar)
    return filtered_words

In [67]:
def filter_not_in_question(answers, question):
    filtered_answers = []
    for answer in answers:
        vp = answer.split('<vp>')[1]
        np = answer.split('<np>')[1]
        if (vp not in question or len(vp)<10) and (np not in question or len(np)<10):
            filtered_answers.append(answer)
    return filtered_answers

In [68]:
def parse_sentence(sentence, recursive = True):
    sent = list(nlp(sentence).sents)[0]
    phrases = get_phrases(sent, recursive = recursive)
    questions, answers = get_QA(phrases)
    qa_pairs = []
    filtered_answers = filter_edit_distance(answers)
    # for question in questions:
    #     filtered_answers = answers #filter_not_in_question(answers, question)
    #     qa_pairs.append({'question':question, 'answers':filtered_answers})
    return [str(sent), questions, filtered_answers, [answer for answer in answers if answer not in filtered_answers], phrases]

In [69]:
if __name__ == '__main__':
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('benepar', config={'model': 'benepar_en3'})
    sentences = pd.read_csv("sentences.csv")
    sentences = sentences.to_numpy().squeeze()
    signs = [',', ':', ';', '.', '!', '?']
    recursive = True


    entries = []
    for idx, sentence in enumerate(sentences):
        entries.append(parse_sentence(sentence, recursive))
        print(f"Finished {idx}/{len(sentences)-1}")
    df = pd.DataFrame(entries, columns =['sentence', 'questions', 'answers','removed', 'phrases'])
    df.to_csv('parsed_sentences.csv')

[[  0 -31 -31 -57 -88 -88 -46 -77 -77 -40 -71 -71 -56 -87 -87 -53 -84 -84
  -60 -91 -91]
 [-31   0  -3 -39 -57 -60 -37 -46 -49 -50 -40 -43 -44 -56 -59 -45 -53 -56
  -47 -60 -63]
 [-31  -3   0 -36 -60 -57 -34 -49 -46 -47 -43 -40 -41 -59 -56 -42 -56 -53
  -44 -63 -60]
 [-57 -39 -36   0 -31 -31 -14 -45 -44 -22 -46 -45  -8 -39 -39  -9 -40 -40
  -13 -44 -44]
 [-88 -57 -60 -31   0  -3 -45 -14 -17 -53 -22 -25 -39  -8 -11 -40  -9 -12
  -44 -13 -16]
 [-88 -60 -57 -31  -3   0 -45 -17 -14 -53 -25 -22 -39 -11  -8 -40 -12  -9
  -44 -16 -13]
 [-46 -37 -34 -14 -45 -45   0 -31 -31 -19 -50 -50 -16 -47 -47 -17 -48 -48
  -18 -49 -49]
 [-77 -46 -49 -45 -14 -17 -31   0  -3 -50 -19 -22 -46 -16 -19 -48 -17 -20
  -49 -18 -21]
 [-77 -49 -46 -44 -17 -14 -31  -3   0 -50 -22 -19 -45 -19 -16 -48 -20 -17
  -49 -21 -18]
 [-40 -50 -47 -22 -53 -53 -19 -50 -50   0 -31 -31 -16 -47 -47 -13 -44 -44
  -20 -51 -51]
 [-71 -40 -43 -46 -22 -25 -50 -19 -22 -31   0  -3 -39 -16 -19 -44 -13 -16
  -51 -20 -23]
 [-71 -43 -40 -45 -25

/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[[  0  -9 -21 ... -32 -43 -41]
 [ -9   0 -12 ... -28 -41 -38]
 [-21 -12   0 ... -19 -45 -44]
 ...
 [-32 -28 -19 ...   0 -32 -29]
 [-43 -41 -45 ... -32   0  -3]
 [-41 -38 -44 ... -29  -3   0]]
Finished 1/22
Finished 2/22


/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[[  0 -30 -21 -21 -51 -42 -22 -39 -35 -19 -49 -39]
 [-30   0 -10 -51 -21 -31 -52 -22 -32 -49 -19 -29]
 [-21 -10   0 -42 -31 -21 -43 -27 -22 -40 -28 -19]
 [-21 -51 -42   0 -30 -21 -27 -40 -39  -7 -37 -28]
 [-51 -21 -31 -30   0 -10 -57 -27 -37 -37  -7 -17]
 [-42 -31 -21 -21 -10   0 -48 -32 -27 -28 -17  -7]
 [-22 -52 -43 -27 -57 -48   0 -30 -21 -22 -52 -43]
 [-39 -22 -27 -40 -27 -32 -30   0 -10 -39 -22 -27]
 [-35 -32 -22 -39 -37 -27 -21 -10   0 -36 -32 -22]
 [-19 -49 -40  -7 -37 -28 -22 -39 -36   0 -30 -21]
 [-49 -19 -28 -37  -7 -17 -52 -22 -32 -30   0 -10]
 [-39 -29 -19 -28 -17  -7 -43 -27 -22 -21 -10   0]]
Finished 3/22
Finished 4/22
Finished 5/22


...

[[  0  -7 -21 -62 -64 -14 -21 -34 -76 -77  -6 -13 -27 -68 -70 -14 -21 -35
  -76 -78]
 [ -7   0 -14 -56 -57 -21 -14 -28 -70 -71 -13  -6 -20 -62 -63 -21 -14 -28
  -70 -71]
 [-21 -14   0 -44 -45 -35 -28 -14 -58 -59 -27 -20  -6 -50 -51 -35 -28 -14
  -58 -59]
 [-62 -56 -44   0  -5 -76 -70 -58 -14 -19 -68 -62 -50  -6 -11 -76 -70 -58
  -14 -19]
 [-64 -57 -45  -5   0 -78 -71 -59 -19 -14 -70 -63 -51 -11  -6 -78 -71 -59
  -19 -14]
 [-14 -21 -35 -76 -78   0  -7 -21 -62 -64 -15 -22 -36 -77 -79 -21 -28 -42
  -83 -85]
 [-21 -14 -28 -70 -71  -7   0 -14 -56 -57 -22 -15 -29 -71 -72 -28 -21 -35
  -77 -78]
 [-34 -28 -14 -58 -59 -21 -14   0 -44 -45 -31 -26 -15 -59 -60 -40 -34 -21
  -65 -66]
 [-76 -70 -58 -14 -19 -62 -56 -44   0  -5 -75 -70 -59 -15 -20 -83 -77 -65
  -21 -26]
 [-77 -71 -59 -19 -14 -64 -57 -45  -5   0 -74 -69 -58 -20 -15 -83 -77 -66
  -26 -21]
 [ -6 -13 -27 -68 -70 -15 -22 -31 -75 -74   0  -7 -21 -62 -64 -14 -21 -35
  -76 -78]
 [-13  -6 -20 -62 -63 -22 -15 -26 -70 -69  -7   0 -14 -56 -57 -21

/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[[  0 -62 -23 ... -42 -40 -38]
 [-62   0 -54 ... -76 -77 -76]
 [-23 -54   0 ... -22 -27 -30]
 ...
 [-42 -76 -22 ...   0  -5  -8]
 [-40 -77 -27 ...  -5   0  -3]
 [-38 -76 -30 ...  -8  -3   0]]
Finished 21/22


/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[[  0  -9 -12  -9 -12  -9 -18 -21 -18 -21 -13 -22 -25 -22 -25 -15 -24 -27
  -24 -27]
 [ -9   0  -3   0  -3 -18  -9 -12  -9 -12 -22 -13 -16 -13 -16 -24 -15 -18
  -15 -18]
 [-12  -3   0  -3   0 -21 -12  -9 -12  -9 -25 -16 -13 -16 -13 -27 -18 -15
  -18 -15]
 [ -9   0  -3   0  -3 -18  -9 -12  -9 -12 -22 -13 -16 -13 -16 -24 -15 -18
  -15 -18]
 [-12  -3   0  -3   0 -21 -12  -9 -12  -9 -25 -16 -13 -16 -13 -27 -18 -15
  -18 -15]
 [ -9 -18 -21 -18 -21   0  -9 -12  -9 -12 -13 -22 -25 -22 -25 -15 -24 -27
  -24 -27]
 [-18  -9 -12  -9 -12  -9   0  -3   0  -3 -22 -13 -16 -13 -16 -24 -15 -18
  -15 -18]
 [-21 -12  -9 -12  -9 -12  -3   0  -3   0 -25 -16 -13 -16 -13 -27 -18 -15
  -18 -15]
 [-18  -9 -12  -9 -12  -9   0  -3   0  -3 -22 -13 -16 -13 -16 -24 -15 -18
  -15 -18]
 [-21 -12  -9 -12  -9 -12  -3   0  -3   0 -25 -16 -13 -16 -13 -27 -18 -15
  -18 -15]
 [-13 -22 -25 -22 -25 -13 -22 -25 -22 -25   0  -9 -12  -9 -12  -6 -15 -18
  -15 -18]
 [-22 -13 -16 -13 -16 -22 -13 -16 -13 -16  -9   0  -3   0  -3 -15

In [70]:
# df['answers'].apply(lambda x:len(x))
# pd_print_entry(df, 21)
print(len(entries[0]))
df = pd.DataFrame(entries, columns =['sentence', 'questions', 'answers','removed', 'phrases'])
#df[['sentence', 'questions', 'answers', 'phrases']]


5


In [71]:
#df = pd.read_csv('parsed_sentences.csv')
parse_sentence(df['sentence'].iloc[8])
# parse_sentence(df['sentence'].iloc[8])

/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[[  0 -21 -18 -23 -44 -41 -10 -31 -28 -13 -34 -31 -10 -31 -28]
 [-21   0  -3 -39 -23 -26 -31 -10 -13 -34 -13 -16 -31 -10 -13]
 [-18  -3   0 -36 -26 -23 -28 -13 -10 -31 -16 -13 -28 -13 -10]
 [-23 -39 -36   0 -21 -18 -17 -30 -30 -12 -33 -30 -23 -38 -37]
 [-44 -23 -26 -21   0  -3 -38 -17 -20 -33 -12 -15 -44 -23 -26]
 [-41 -26 -23 -18  -3   0 -35 -20 -17 -30 -15 -12 -41 -26 -23]
 [-10 -31 -28 -17 -38 -35   0 -21 -18 -13 -34 -31  -9 -30 -27]
 [-31 -10 -13 -30 -17 -20 -21   0  -3 -34 -13 -16 -30  -9 -12]
 [-28 -13 -10 -30 -20 -17 -18  -3   0 -31 -16 -13 -27 -12  -9]
 [-13 -34 -31 -12 -33 -30 -13 -34 -31   0 -21 -18 -13 -34 -31]
 [-34 -13 -16 -33 -12 -15 -34 -13 -16 -21   0  -3 -34 -13 -16]
 [-31 -16 -13 -30 -15 -12 -31 -16 -13 -18  -3   0 -31 -16 -13]
 [-10 -31 -28 -23 -44 -41  -9 -30 -27 -13 -34 -31   0 -21 -18]
 [-31 -10 -13 -38 -23 -26 -30  -9 -12 -34 -13 -16 -21   0  -3]
 [-28 -13 -10 -37 -26 -23 -27 -12  -9 -31 -16 -13 -18  -3   0]]


/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/sklearn/cluster/_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(


['If a duplicate exists, an error of NFS4ERR_EXIST is returned as the status.',
 ['What If a duplicate exists?'],
 ['<np>the status<np> <vp>exists<vp>',
  '<np>the status<np> <vp>returned as the status<vp>',
  '<np>an error of NFS4ERR_EXIST<np> <vp>returned as the status<vp>',
  '<np>NFS4ERR_EXIST<np> <vp>exists<vp>',
  '<np>NFS4ERR_EXIST<np> <vp>returned as the status<vp>'],
 ['<np>a duplicate<np> <vp>exists<vp>',
  '<np>a duplicate<np> <vp>is returned as the status<vp>',
  '<np>a duplicate<np> <vp>returned as the status<vp>',
  '<np>an error of NFS4ERR_EXIST<np> <vp>exists<vp>',
  '<np>an error of NFS4ERR_EXIST<np> <vp>is returned as the status<vp>',
  '<np>an error<np> <vp>exists<vp>',
  '<np>an error<np> <vp>is returned as the status<vp>',
  '<np>an error<np> <vp>returned as the status<vp>',
  '<np>NFS4ERR_EXIST<np> <vp>is returned as the status<vp>',
  '<np>the status<np> <vp>is returned as the status<vp>'],
 {'SBAR': [If a duplicate exists],
  'S': [a duplicate exists],
  'NP': [